In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import numpy as np
from pathlib import Path

# Add the parent directory to sys.path to access src
sys.path.append(str(Path(os.getcwd()).parent))

from src.color_code_stim import ColorCode

In [28]:
cc = ColorCode(d=3, rounds=3, shape="tri", p_circuit=1e-3, comparative_decoding=True)
det, obs = cc.sample(10000)
preds = cc.decode(det, erasure_matcher_predecoding=True, verbose=True)
print()
print("# failures:", preds.sum())

First-round decoding:
    > logical class 0, color r...
    > logical class 0, color g...
    > logical class 0, color b...
    > logical class 1, color r...
    > logical class 1, color g...
    > logical class 1, color b...
Erasure matcher predecoding:
    > # of samples with successful predecoding: 7914
Second-round decoding:
    > logical class 0, color r...
    > logical class 0, color g...
    > logical class 0, color b...
    > logical class 1, color r...
    > logical class 1, color g...
    > logical class 1, color b...
Merging predecoding & second-round decoding outcomes

# failures: 491


In [20]:
cc = ColorCode(d=3, rounds=3, shape="tri", p_circuit=3e-3, comparative_decoding=True)

det, obs = cc.sample(100_000)
preds1, extra_outputs1 = cc.decode(
    det, erasure_matcher_predecoding=False, full_output=True
)
nfails1 = np.sum(preds1 != obs)
preds2, extra_outputs2 = cc.decode(
    det, erasure_matcher_predecoding=True, full_output=True
)
nfails2 = np.sum(preds2 != obs)
# preds3, extra_outputs3 = cc.decode(
#     det,
#     erasure_matcher_predecoding=True,
#     full_output=True,
#     partial_correction_by_predecoding=True,
# )
# nfails3 = np.sum(preds3 != obs)

print("Original: ")
print(nfails1)
print("Erasure matcher: ")
print(nfails2)
# print('Erasure matcher with partial correction: ')
# print(nfails3)

Original: 
5189
Erasure matcher: 
5013


In [21]:
N = round(1e6)
cc = ColorCode(d=5, rounds=1, shape="tri", p_bitflip=1e-2, comparative_decoding=True)
preds1 = cc.simulate(N)
preds2 = cc.simulate(N, erasure_matcher_predecoding=True)
print(preds1)
print(preds2)

540
507
